# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [53]:
# import required libraries
import numpy as np
import pandas as pd

In [54]:
import zipfile

In [55]:
from zipfile import ZipFile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [56]:
zf = zipfile.ZipFile('Orders.zip')
orders =  pd.read_csv(zf.open('Orders.csv'))

In [57]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [58]:
# How to aggregate the amount_spent for unique customers?

customer_spent = orders.groupby(['CustomerID','Country']).agg({'amount_spent':sum})
customer_spent.head()


,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40


In [59]:
# How to select customers whose aggregated amount_spent is in a given quantile range?

In [60]:
qlabels = ["Q1", "Q2", "Q3", "Q4", "Q5"]
quantiles = pd.cut(customer_spent['amount_spent'], 5, labels = qlabels)
quantiles.value_counts()


Q1    4330
Q2      10
Q3       3
Q4       2
Q5       2
Name: amount_spent, dtype: int64

In [61]:
# How to label selected customers as "VIP" or "Preferred"? above the 95th percentile 

In [62]:
customer_spent['amount_spent'].describe([0.75,0.95])

count      4347.000000
mean       2050.013320
std        8980.103578
min           0.000000
50%         671.960000
75%        1661.195000
95%        5774.278000
max      280206.020000
Name: amount_spent, dtype: float64

In [63]:
# VIP CUSTOMERS: spent > 5840.18 
# Preferred: spent [1661.19, 5840.18] 
# Normal: Else 



Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [64]:
# First create the status:

def status(x):
    if x['amount_spent'] >= 5840:
        return 'VIP'
    elif 1661 < x['amount_spent'] < 5840:
        return 'Preferred'
    else:
        return 'Normal'

customer_spent['status'] = customer_spent.apply(status, axis=1)
customer_spent.head()
         

,,amount_spent,status
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,Preferred
12348,Finland,1797.24,Preferred
12349,Italy,1757.55,Preferred
12350,Norway,334.40,Normal


In [65]:
customer_status_country = customer_spent.groupby(['Country','status']).agg({'amount_spent':sum}).reset_index()
customer_status_country .head()

,Country,status,amount_spent
0,Australia,Normal,3550.49
1,Australia,Preferred,10056.29
2,Australia,VIP,124914.53
3,Austria,Normal,7536.62
4,Austria,Preferred,2662.06


In [69]:
customer_status_country[customer_status_country['status'] == 'VIP'].sort_values(by=['amount_spent'],ascending=False)

,Country,status,amount_spent
73,United Kingdom,VIP,3417798.95
48,Netherlands,VIP,280206.02
22,EIRE,VIP,261204.69
2,Australia,VIP,124914.53
29,France,VIP,106383.09
32,Germany,VIP,100587.33
65,Sweden,VIP,31906.82
42,Japan,VIP,28406.43
68,Switzerland,VIP,26315.86
62,Spain,VIP,25391.20


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [76]:
customers_not_normal = customer_status_country[customer_status_country['status'] != 'Normal']
customers_not_normal.head()

,Country,status,amount_spent
1,Australia,Preferred,10056.29
2,Australia,VIP,124914.53
4,Austria,Preferred,2662.06
7,Belgium,Preferred,31184.22
10,Canada,Preferred,2940.04


In [79]:
customers_not_normal.groupby('Country').agg({'amount_spent':sum}).reset_index().sort_values(by=['amount_spent'],ascending=False)

,Country,amount_spent
27,United Kingdom,5627835.481
19,Netherlands,280206.020
7,EIRE,265545.900
10,Germany,194078.540
9,France,174694.670
0,Australia,134970.820
24,Spain,46564.700
26,Switzerland,45381.990
15,Japan,35229.120
20,Norway,35065.630
